In [1]:
import networkx as nx
import random
import copy
from tqdm.notebook import tqdm

## Create

In [2]:
amazon_G = nx.read_graphml('./amazon_graph.graphml')

# Minimizar la distancia entre los nodos dentro de una comunidad
# Maximizar la distancia entre los nodos de disintas comunidades

In [3]:
def create_individuo(nodes_of_graph, adj_matrix, transformation):
    individuo =  [None] * len(nodes_of_graph)
    for i in range(len(nodes_of_graph)): #el individuo creado con el tipo locus tiene la longitud de los nodos

        
        nodo_vecino = random.choice(range(len(adj_matrix[i]))) #la matriz de adyacencia tiene los nodos vecinos, los nodos vecinos hay que convertirlos a numeros
        while adj_matrix[i][nodo_vecino] == 0 and nodo_vecino != i:
            nodo_vecino = random.choice(range(len(adj_matrix[nodo_vecino])))
        
        
        keys = list(transformation.keys())
        values = list(transformation.values())
        individuo[i] = keys[values.index(nodo_vecino)] # en el gen se pone el aleatorio de los vecinos pero transformado en el indice que le corresponde 
    return individuo

def create(nodes_of_graph, adj_matrix, transformation, N=100): # crea población, la transformacion debe de ser del tipo { numero_de_nodo_real : indice_del_array}, donde el indice del array va de 0 a len(nodos) - 1
    new_population = []
    for individuo in range(N):
        new_population.append(create_individuo(nodes_of_graph, adj_matrix, transformation))
    
    return new_population

In [4]:
conversiones = {}
for i, nodo in enumerate(amazon_G.nodes):
    conversiones[nodo] = i
print(conversiones)

{'75780': 0, '317458': 1, '186386': 2, '419862': 3, '176159': 4, '204831': 5, '104481': 6, '57383': 7, '401449': 8, '299055': 9, '176184': 10, '542780': 11, '366654': 12, '124992': 13, '16457': 14, '196682': 15, '546890': 16, '546891': 17, '34895': 18, '321619': 19, '112724': 20, '276567': 21, '55386': 22, '374874': 23, '55388': 24, '221281': 25, '432229': 26, '104553': 27, '383087': 28, '335987': 29, '448649': 30, '522378': 31, '59532': 32, '491667': 33, '518306': 34, '309411': 35, '182435': 36, '204966': 37, '532649': 38, '544939': 39, '542899': 40, '172218': 41, '270524': 42, '250046': 43, '204991': 44, '231620': 45, '497867': 46, '186575': 47, '346322': 48, '399572': 49, '276695': 50, '530647': 51, '18650': 52, '252125': 53, '59613': 54, '127199': 55, '379102': 56, '159972': 57, '366824': 58, '116969': 59, '202986': 60, '430314': 61, '350442': 62, '4328': 63, '458993': 64, '30962': 65, '35067': 66, '57600': 67, '217346': 68, '295177': 69, '49424': 70, '278': 71, '123159': 72, '3218

In [5]:
A = nx.to_numpy_array(amazon_G)

In [6]:
pop = create(amazon_G.nodes, A, conversiones)

## Fitness

In [7]:
def convert_individuo(individuo_raw, conversiones):
    new_individuo = [None] * len(individuo_raw)
    for i in range(len(individuo_raw)):
        new_individuo[i] = conversiones.get(str(individuo_raw[i]))
    
    return new_individuo
def unconvert_individuo(individuo, conversiones):
    individuo_raw =  [None] * len(individuo)
    keys = list(conversiones.keys())
    values = list(conversiones.values())
    for i in range(len(individuo)):
        individuo_raw[i] = keys[values.index(individuo[i])]
    return individuo_raw

def get_comunidades(individuo):

    G = nx.Graph()

    for i in range(len(individuo)):
        G.add_edge(i, individuo[i])

    comunidades = list(nx.connected_components(G))
    return comunidades

def get_cs(aristas_of_min_one_node_in_comunity, nodos_de_la_comunidad):
    cs = 0
    for aristas in aristas_of_min_one_node_in_comunity:
        if aristas[0] in nodos_de_la_comunidad and aristas[1] not in nodos_de_la_comunidad:
            cs += 1
            
        #elif aristas[1] in nodos_de_la_comunidad and aristas[0] not in nodos_de_la_comunidad:
        #    cs += 1
    return cs

def get_ms(aristas_of_min_one_node_in_comunity, nodos_de_la_comunidad):
    ms = 0
    for aristas in aristas_of_min_one_node_in_comunity:
        if aristas[0] in nodos_de_la_comunidad and aristas[1] in nodos_de_la_comunidad:
            ms += 1
            
        
    return ms

In [8]:
def fit_conductance(grafo,individuo,conversiones):
    conductance = 0
    individuo_converted = convert_individuo(individuo, conversiones)
    comunidades = get_comunidades(individuo_converted)
    for comunidad in comunidades:
        aristas_of_min_one_node_in_comunity = []
        comunidad_desconvertida = unconvert_individuo(list(comunidad), conversiones)
        for nodo_of_comunity in comunidad_desconvertida:

            aristas_of_min_one_node_in_comunity.append(list(grafo.edges(str(nodo_of_comunity)))[0])
        #aqui ya tenemos las aristas de los nodos para poder sacar el sm
        cs_community = get_cs(aristas_of_min_one_node_in_comunity, comunidad_desconvertida)
        ms_community = get_ms(aristas_of_min_one_node_in_comunity, comunidad_desconvertida)
        conductance += cs_community / (2*ms_community + cs_community)
        #
    
    return conductance
        
        
def fit_expansion(grafo,individuo,conversiones):
    expansion = 0
    individuo_converted = convert_individuo(individuo, conversiones)
    comunidades = get_comunidades(individuo_converted)
    for comunidad in comunidades:
        aristas_of_min_one_node_in_comunity = []
        comunidad_desconvertida = unconvert_individuo(list(comunidad), conversiones)
        for nodo_of_comunity in comunidad_desconvertida:

            aristas_of_min_one_node_in_comunity.append(list(grafo.edges(str(nodo_of_comunity)))[0])
        #aqui ya tenemos las aristas de los nodos para poder sacar el sm
        cs_community = get_cs(aristas_of_min_one_node_in_comunity, comunidad_desconvertida)
        ns_community = len(comunidad_desconvertida)
        expansion += cs_community / ns_community
        #
    
    return expansion


def fit_average_odf(grafo,individuo,conversiones):
    average_odf = 0
    individuo_converted = convert_individuo(individuo, conversiones)
    comunidades = get_comunidades(individuo_converted)
    for comunidad in comunidades:
        comunidad_desconvertida = unconvert_individuo(list(comunidad), conversiones)
        sum_nodos = 0
        for nodo_of_comunity in comunidad_desconvertida:
            aristas_del_nodo = list(grafo.edges(str(nodo_of_comunity)))[0]
            
            sum_nodos += get_cs(aristas_del_nodo,nodo_of_comunity) / len(aristas_del_nodo)
            #aristas_of_min_one_node_in_comunity.append(list(grafo.edges(str(nodo_of_comunity)))[0])
        average_odf += sum_nodos / len(comunidad)
        
        #aqui ya tenemos las aristas de los nodos para poder sacar el sm
        
        #
    
    return average_odf

def fit_Q(grafo,individuo,conversiones):
    Q = 0
    individuo_converted = convert_individuo(individuo, conversiones)
    comunidades = get_comunidades(individuo_converted)
    for comunidad in comunidades:
        aristas_of_min_one_node_in_comunity = []
        comunidad_desconvertida = unconvert_individuo(list(comunidad), conversiones)
        n_aristas = 0
        for nodo_of_comunity in comunidad_desconvertida:
            n_aristas += len(list(grafo.edges(str(nodo_of_comunity)))[0])
            aristas_of_min_one_node_in_comunity.append(list(grafo.edges(str(nodo_of_comunity)))[0])
        
        ms_community = get_ms(aristas_of_min_one_node_in_comunity, comunidad_desconvertida)
        cs_community = get_cs(aristas_of_min_one_node_in_comunity, comunidad_desconvertida)
        
        Q += ((ms_community/n_aristas) - (((ms_community + cs_community)/(2*n_aristas))**2))
    
    return Q

## Select, Mutate y Crossover

In [9]:
def select (pop, fits, T): # devuelve un individuo seleccionado por torneo, devuelve una copia para evitar efectos laterales
    seleccionados = random.sample(pop, T) #los 3 aleatorios
    es_dominante = True
    for i, indiv in enumerate(seleccionados):
        for j, indiv2 in enumerate(seleccionados): 
            if i == j:
                continue
            
            if fits[i][0] > fits[j][0] and fits[i][1] > fits[j][1]:
                es_dominante = (True and es_dominante)
            else:
                es_dominante = (False and es_dominante)
        if es_dominante:
            return indiv
    return random.choice(seleccionados)

In [10]:
def mutate (grafo, ind, pmut, transformation): # la mutación consistirá en cambiar un elemento por otro posible nodo con el que este conectado
    probs = [random.random() for prob in range(len(ind))]
    nuevo_elem = copy.copy(ind)
    adj_matrix = nx.to_numpy_array(grafo)
    for i, probability in enumerate(probs):
      if probability < pmut:
            
        nodo_vecino = random.choice(range(len(adj_matrix[i]))) #la matriz de adyacencia tiene los nodos vecinos, los nodos vecinos hay que convertirlos a numeros
        
        while adj_matrix[i][nodo_vecino] == 0 and nodo_vecino != i:
            nodo_vecino = random.choice(range(len(adj_matrix[nodo_vecino])))
        
        keys = list(transformation.keys())
        values = list(transformation.values())
        nuevo_elem[i] = keys[values.index(nodo_vecino)] # en el gen se pone el aleatorio de los vecinos pero transformado en el indice que le corresponde 
    return nuevo_elem

In [11]:
def crossover (ind1, ind2, pcross, n_puntos = 1):
    copia_in1 = copy.copy(ind1)
    copia_in2 = copy.copy(ind2)
    if random.random() < pcross:
      punto = len(ind1) // (n_puntos+1)
      for i in range(n_puntos):
        index_fin = punto*(i+2)
        index_inicio = punto*(i+1)
        if index_fin < len(ind1):
          aux = copia_in1[index_inicio:index_fin]
          copia_in1[index_inicio:index_fin] = copia_in2[index_inicio:index_fin]
          copia_in2[index_inicio:index_fin] = aux
        else:
          aux = copia_in1[index_inicio:]
          copia_in1[index_inicio:] = copia_in2[index_inicio:]
          copia_in2[index_inicio:] = aux
    return copia_in1, copia_in2

# Evolve

In [12]:
def dominates(fit1, fit2):
    return all(f1 <= f2 for f1, f2 in zip(fit1, fit2)) and any(f1 < f2 for f1, f2 in zip(fit1, fit2))

def get_pareto_front(pop, fits):
    pareto_front = []
    for i, fit in enumerate(fits):
        if not any(dominates(fit, fits[j]) for j in range(len(pop)) if i != j):
            pareto_front.append(i)
    return pareto_front

def select_pareto(pop, fits, pareto_front, n):
    pareto_pop = [pop[i] for i in pareto_front]
    pareto_fits = [fits[i] for i in pareto_front]

    # Ordenar soluciones no dominadas según la diversidad de objetivos
    sorted_indices = sorted(range(len(pareto_pop)), key=lambda k: pareto_fits[k])

    # Seleccionar las mejores soluciones no dominadas
    selected_indices = sorted_indices[:n]

    return [pareto_pop[i] for i in selected_indices]

In [29]:
def evolve(pop, fit1, fit2, grafo, conversiones, T=3, pcross=0.8, pmut=0.2, n_puntos=1, n_gen=10, fit1_maximize=False, fit2_maximize=False):
    fitness1 = fit1 if fit1_maximize else lambda grafo, ind, conversione: 1 / (fit1(grafo, ind, conversione) + 0.00000000001)
    fitness2 = fit2 if fit2_maximize else lambda grafo, ind, conversione: 1 / (fit2(grafo, ind, conversione) + 0.00000000001)
    
    for _ in tqdm(range(n_gen)):
        new_pop = []

        fits = [(fitness1(grafo, ind, conversiones), fitness2(grafo, ind, conversiones)) for ind in pop]

        for _ in range(len(pop) // 2):
            parent1 = select(pop, fits, T)
            parent2 = select(pop, fits, T)

            child1, child2 = crossover(parent1, parent2, pcross, n_puntos)

            child1 = mutate(grafo, child1, pmut, conversiones)
            child2 = mutate(grafo, child2, pmut, conversiones)

            new_pop.extend([child1, child2])

        combined_pop = pop + new_pop
        combined_fits = [(fit_conductance(grafo, ind, conversiones), fit_expansion(grafo, ind, conversiones),
                          fit_average_odf(grafo, ind, conversiones), fit_Q(grafo, ind, conversiones)) for ind in combined_pop]

        pareto_front = get_pareto_front(combined_pop, combined_fits)

        pop = select_pareto(combined_pop, combined_fits, pareto_front, len(pop))

    return pop


In [30]:
evolved_pop = evolve(pop, fit_conductance, fit_expansion, amazon_G, conversiones, n_gen=100, fit2_maximize=True)
evolved_pop

  0%|          | 0/100 [00:00<?, ?it/s]

[['458993',
  '480479',
  '414657',
  '419862',
  '404948',
  '379184',
  '394886',
  '165463',
  '401449',
  '186386',
  '256445',
  '542899',
  '366654',
  '124992',
  '16457',
  '216152',
  '246060',
  '123181',
  '542633',
  '365354',
  '117287',
  '80756',
  '55386',
  '33710',
  '55388',
  '202986',
  '392463',
  '104553',
  '289953',
  '27890',
  '52785',
  '329078',
  '521655',
  '491667',
  '518306',
  '541690',
  '446298',
  '120615',
  '500861',
  '482998',
  '542899',
  '203272',
  '132756',
  '250046',
  '58975',
  '231620',
  '497867',
  '423306',
  '346322',
  '219603',
  '58977',
  '530647',
  '401165',
  '401798',
  '35067',
  '127199',
  '379102',
  '189286',
  '366824',
  '141618',
  '221281',
  '430314',
  '476448',
  '4328',
  '152407',
  '30962',
  '59613',
  '525489',
  '217346',
  '30962',
  '385399',
  '278',
  '123159',
  '321818',
  '480479',
  '221281',
  '85382',
  '500473',
  '525489',
  '313954',
  '151849',
  '330328',
  '270524',
  '546890',
  '190765',

In [32]:
get_comunidades(evolved_pop[0])

[{0, 218, 220, 310, '458993'},
 {1, 300, '480479', 74},
 {2, '414657'},
 {205, 3, '419862'},
 {342, 4, '404948'},
 {'379184', 5},
 {242, '394886', 6},
 {'165463', 7},
 {'401449', 8},
 {'186386', 9},
 {10, '256445'},
 {11, 386, 40, '542899'},
 {12, '366654', 464},
 {'124992', 13},
 {14, '16457', 268, 398},
 {15, '216152'},
 {16, '246060', 408},
 {'123181', 17},
 {18, 307, '542633'},
 {19, '365354'},
 {'117287', 20},
 {21, 216, '80756'},
 {104, 207, 22, '55386'},
 {226, 23, '33710'},
 {24, '55388'},
 {'202986', 245, 25},
 {26, '392463'},
 {'104553', 27, 471},
 {232, 28, '289953', 463},
 {'27890', 29, 379},
 {122, 30, '52785'},
 {196, 31, '329078'},
 {164, 32, 447, '521655'},
 {33, '491667'},
 {197, 34, '518306'},
 {35, '541690'},
 {36, 444, '446298'},
 {'120615', 37},
 {219, 274, 38, '500861'},
 {325, 39, 412, '482998'},
 {133, '203272', 281, 41, 451},
 {'132756', 42},
 {'250046', 43},
 {44, '58975'},
 {'231620', 293, 45},
 {285, 46, '497867'},
 {'423306', 47},
 {'346322', 48},
 {124, '2